In [1]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)
    .master("local[*]")
    .getOrCreate()
)

spark

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/09 05:30:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
from pyspark.sql.functions import udf, from_unixtime, year, weekofyear, row_number
from pyspark.sql.functions import hour, minute, countDistinct, col, sum, count, to_date, concat, lit, when,round
from pyspark.sql.window import Window

from pyspark.sql.types import DoubleType

In [2]:
# df = spark.read.parquet("s3://zarklab-hive-test/buy-sell-updated")
data_raw = spark.read.parquet('s3a://zarklab-hive-test/buy-sell-updated/pt_d=202202*/*')


NameError: name 'spark' is not defined

In [7]:
data_raw.show(5)

+-------+----------+-----------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+------------------+
|chainId| timestamp|blockNumber|transactionIndex|     transactionHash|               buyer|           recipient|           txFee_eth|     exchangeAddress|exchangeName|            buyAsset|              buyQty|             buyQty1|          buyPrice|           sellAsset|             sellQty|            sellQty1|         sellPrice|
+-------+----------+-----------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+------------------+
|   

In [8]:
# side = spark.read.option("header",True).csv("token_recommender_v11.csv")

In [9]:
# side.show(3)

In [14]:
data_raw.columns

['chainId',
 'timestamp',
 'blockNumber',
 'transactionIndex',
 'transactionHash',
 'buyer',
 'recipient',
 'txFee_eth',
 'exchangeAddress',
 'exchangeName',
 'buyAsset',
 'buyQty',
 'buyQty1',
 'buyPrice',
 'sellAsset',
 'sellQty',
 'sellQty1',
 'sellPrice']

In [15]:
data = data_raw.select(['timestamp','recipient','txFee_eth','buyAsset','buyQty1','buyPrice'])

In [16]:
data.show(5)

+----------+--------------------+--------------------+--------------------+--------------------+------------------+
| timestamp|           recipient|           txFee_eth|            buyAsset|             buyQty1|          buyPrice|
+----------+--------------------+--------------------+--------------------+--------------------+------------------+
|1644411457|0xd5f20543784f105...| 0.01026891574870283|0x000000000000000...|  0.3657743824746011|1162.8619784799514|
|1644382371|0x73f761af1e83bf5...|0.018718289361991356|0x74edaf28fc4b9e6...|2.077444376595765...|141.77016194281217|
|1644427655|0x32b466947abbda8...|0.011949448357737216|0xc02aaa39b223fe8...|                 1.0|   3209.1297375821|
|1644371711|0x9397008382f8c69...|0.008915766297347447|0x16756ec1deb89a2...|  121532.57015931424| 4651.701271582571|
|1644421478|0xcad1f42d720fa16...| 0.02433959936234635|0x000000000000000...|  1.3085875477648865|  4155.14657456207|
+----------+--------------------+--------------------+------------------

In [19]:
data_ = data.withColumn("timestamp", from_unixtime(col("timestamp")))


In [20]:
data_.show(5)

+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|          timestamp|           recipient|           txFee_eth|            buyAsset|             buyQty1|          buyPrice|
+-------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|2022-02-09 12:57:37|0xd5f20543784f105...| 0.01026891574870283|0x000000000000000...|  0.3657743824746011|1162.8619784799514|
|2022-02-09 04:52:51|0x73f761af1e83bf5...|0.018718289361991356|0x74edaf28fc4b9e6...|2.077444376595765...|141.77016194281217|
|2022-02-09 17:27:35|0x32b466947abbda8...|0.011949448357737216|0xc02aaa39b223fe8...|                 1.0|   3209.1297375821|
|2022-02-09 01:55:11|0x9397008382f8c69...|0.008915766297347447|0x16756ec1deb89a2...|  121532.57015931424| 4651.701271582571|
|2022-02-09 15:44:38|0xcad1f42d720fa16...| 0.02433959936234635|0x000000000000000...|  1.3085875477648865|  4155.14657456207|


In [21]:
data_dropped = data_.na.drop()

In [22]:
data_dropped.count()

1352297

In [23]:
data_dropped.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- recipient: string (nullable = true)
 |-- txFee_eth: double (nullable = true)
 |-- buyAsset: string (nullable = true)
 |-- buyQty1: string (nullable = true)
 |-- buyPrice: string (nullable = true)



In [24]:
data_dropped = data_dropped.withColumn("buyQty1",round(data_dropped.buyQty1.cast(DoubleType()),2))
data_dropped = data_dropped.withColumn("buyPrice",round(data_dropped.buyPrice.cast(DoubleType()),2))


In [25]:
data_dropped.write.mode("overwrite").parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202202/")


In [11]:
data_.select("recipient").distinct().count()

415953

In [12]:
data_.select("buyAsset").distinct().count()

4823

# filter out unseen tokens

In [26]:
train_data  = spark.read.parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202201/")


In [27]:
train_tokens = train_data.select("buyAsset").distinct()

In [28]:
train_tokens.count()

4688

In [29]:
eva_data  = spark.read.parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202202/")


In [30]:
eval_tokens = eva_data.select("buyAsset").distinct()

In [31]:
eval_tokens.count()

5410

In [36]:
eva_data_filter = eva_data.join(train_tokens, on="buyAsset", how="inner")


In [37]:
eva_data_filter.select("buyAsset").distinct().count()

2787

In [38]:
eva_data_filter.write.mode("overwrite").parquet(f"s3a://zarklab-hive-test/buy-sell-updated-202202_filter/")


In [35]:
eva_data_filter.show()

+--------------------+-------------------+--------------------+--------------------+--------------+--------+
|            buyAsset|          timestamp|           recipient|           txFee_eth|       buyQty1|buyPrice|
+--------------------+-------------------+--------------------+--------------------+--------------+--------+
|0x00000006e55a936...|               null|                null|                null|          null|    null|
|0x044727e50ff30db...|2022-02-09 00:10:50|0x669735809ff5902...|0.025893602333315245| 3.769510099E7| 1185.38|
|0x044727e50ff30db...|2022-02-12 23:21:59|0xb51f25212ebd2fd...|0.013157703921105763| 1.818520453E7|   435.2|
|0x044727e50ff30db...|2022-02-12 00:40:43|0xb8eee1ae26a0722...| 0.01790774287106544| 6.622289934E7|  1550.5|
|0x044727e50ff30db...|2022-02-11 16:06:59|0xba15cfb397dcaf6...|0.021735606853078057|         1.0E8| 2560.22|
|0x044727e50ff30db...|2022-02-12 17:33:29|0xd73619f498afec2...| 0.01533242454091782|    4591549.52|  102.43|
|0x044727e50ff30db.